In [58]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE

import os
import sys
PROJECT_ROOT = os.path.abspath( # Getting Obtaining the absolute normalized version of the project root path (Obtendo a versão absoluta normalizada do path raíz do projeto)
    os.path.join( # Concatenating the paths (Concatenando os paths)
        os.getcwd(), # # Getting the path of the notebooks directory (Obtendo o path do diretório dos notebooks)
        os.pardir # Gettin the constant string used by the OS to refer to the parent directory (Obtendo a string constante usada pelo OS para fazer referência ao diretório pai)
    )
)
# Adding path to the list of strings that specify the search path for modules
# Adicionando o path à lista de strings que especifica o path de pesquisa para os módulos
sys.path.append(PROJECT_ROOT)
from src.preprocessing import *

In [26]:
files = ['train', 'valid', 'test']
datasets = []
for file in files:
    with open(f'../data/preprocessed/{file}_corpus.npy', 'rb') as f:
        datasets.append(np.load(f))

train_corpus, valid_corpus, test_corpus = datasets
print(f'Train set shape: {train_corpus.shape}\nValidation set shape: {valid_corpus.shape}\nTest set shape: {test_corpus.shape}')

Train set shape: (9682, 167)
Validation set shape: (3227, 167)
Test set shape: (3228, 167)


In [28]:
BATCH_SIZE = 128
BUFFER_SIZE = 1000

In [56]:
def create_batch_dataset(dataset, batch_size=64, buffer_size=10000):
    """
    
    """
    dataset_final = Dataset.from_tensor_slices((dataset[:, :-1], dataset[:, -1]))
    dataset_final = (dataset_final
                    .batch(batch_size)
                    .shuffle(buffer_size)
                    .prefetch(AUTOTUNE)
                    .cache()
                    )

    return dataset_final

In [54]:
train_set = create_batch_dataset(train_corpus, BATCH_SIZE, BUFFER_SIZE)
valid_set = create_batch_dataset(valid_corpus, BATCH_SIZE, BUFFER_SIZE)
test_set = create_batch_dataset(test_corpus, BATCH_SIZE, BUFFER_SIZE)

type(train_set)

tensorflow.python.data.ops.cache_op.CacheDataset